In [6]:
%env STRAPI_HOSTS=http://13.124.235.216:1337/
%env ES_HOSTS=https://search-deepsearch-bztxmrzp24xb4ms6tddde7vypm.ap-northeast-2.es.amazonaws.com
%env ES_ID=deepsearch
%env ES_PASSWD=Lina123$

env: STRAPI_HOSTS=http://13.124.235.216:1337/
env: ES_HOSTS=https://search-deepsearch-bztxmrzp24xb4ms6tddde7vypm.ap-northeast-2.es.amazonaws.com
env: ES_ID=deepsearch
env: ES_PASSWD=Lina123$


In [7]:
from utils import Get_Article_Body, ariticle_reply, get_article_df
import datetime
from selenium import webdriver
import json
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('DeepSearch'))))
import requests
import json

In [22]:
!echo $ES_ID

deepsearch


In [18]:
os.environ()

TypeError: '_Environ' object is not callable

In [14]:
print(os.getenv('ES_HOSTS'))

https://search-deepsearch-bztxmrzp24xb4ms6tddde7vypm.ap-northeast-2.es.amazonaws.com


In [17]:
print(os.environ.get('ES_HOSTS'))

https://search-deepsearch-bztxmrzp24xb4ms6tddde7vypm.ap-northeast-2.es.amazonaws.com


In [8]:
from elasticsearch import Elasticsearch

In [9]:
from connector import strapi_connector

In [10]:
from connector import es_connector

In [11]:
es = es_connector.ES()

In [12]:
es.allIndex()

'yellow open klue-mrc-dev                    EKq-LljHQmyceeV4R9LVRw 5 1  5076  0  19.6mb  19.6mb\nyellow open test                            _RtSuOgYQG-4uG9J5ulDNA 5 1     0  0     1kb     1kb\ngreen  open .kibana_3599307_user_1          4hlmTMHSQXa35CeNSosAzQ 1 0     1  0   5.2kb   5.2kb\nyellow open news-naver                      cMYtMNzDQOe37VN606U98g 5 1   185  0   2.7mb   2.7mb\nyellow open crawling                        3AFnvNLvR2WzhDN8kFW9PQ 5 1    40  0 381.6kb 381.6kb\ngreen  open .kibana_-700776044_deepsearch_1 87d-qHhWRlqV1KivHOsUxg 1 0    44 10  65.4kb  65.4kb\nyellow open dailynews-naver                 8W52G-MPSki4kQDfloy-YQ 5 1   228  0   3.5mb   3.5mb\ngreen  open .kibana_1                       lvca5kdxTb2Y1LVm1NCPZg 1 0   139 23  56.5kb  56.5kb\ngreen  open .opendistro_security            VFXXzkbeQGqDMR7T2JDSSg 1 0     9  7  56.1kb  56.1kb\ngreen  open .opendistro-reports-instances   tovqnZnqSi2Y1iEbyoFBig 1 0     1  0   6.4kb   6.4kb\nyellow open klue-mrc-train   

In [6]:
strapi = strapi_connector.Strapi()

In [7]:
strapi.hosts

'http://13.124.235.216:1337/'

In [2]:
# 검색 키워드 
strapi = strapi_connector.Strapi()
keywords_db = strapi.get_db()
search_keywords = [K['Keywords'] for K in keywords_db]

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
#chrome driver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome("chromedriver", options = chrome_options)

# 기간 설정
today = datetime.datetime.now()
yesterday = today + datetime.timedelta(days=-1)
years = [yesterday.strftime("%Y-%m-%d")[:4]]
months = [yesterday.strftime("%Y-%m-%d")[5:7]]
all_days = [yesterday.strftime("%Y-%m-%d")[-2:]]
article_url_df = get_article_df(years, months, all_days, search_keywords, driver)
urls = article_url_df['url']

#기사 내용 수집
article_dicts = []
reply_dfs = []

#ElasticSearch
es = es_connector.ES()
index_name = 'dailynews-naver'
# from utils import es_schema
# settings = es_schema.settings
# mappings = es_schema.mappings
# es.createIndex(index=index_name,settings=settings, mappings=mappings)
for i in range(len(article_url_df)):
    try:
        article_dict = Get_Article_Body(article_url_df['url'][i],driver)
        article_dict['토픽'] = article_url_df['search_keyword'][i]
        #print(article_dict)
        es.dataInsert(index=index_name, data=article_dict)
        #reply_df = ariticle_reply()
        #reply_dfs.append(reply_df)
    except:
        continue